# HW_2 

В этой домашке вам будет нужно найти тексты на русском языке, в которых будут какие-то трудные или неоднозначные для POS теггинга моменты и разметить их вручную – с помощью этих текстов мы будем оценивать качество работы наших теггеров. В текстах размечаем только части речи, ничего больше!

## WARNING
**Я дурочка с переулочка, невнимательно прочитала задание, поэтому размечала не только POS, но и еще род, число, время и т. п.**

**Поэтому у меня для 3 пункта будет 2 функции: одна – по заданию – для подсчета accuracy pos, а другая – расширенная – для сравнения всех тегов (время, род, число и т. п.)**


- ✅ (1 балл) Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки (какие моменты вы тут считаете трудными для автоматического посттеггинга и почему). Не забывайте, что разные теггеры могут использовать разные тегсеты: напишите комментарий о том, какой тегсет вы берёте для разметки и почему.
- ✅ (3 балла) Потом вам будет нужно взять три POS теггера для русского языка (**udpipe**, **natasha**, **pymorphy**) и «прогнать» текст через каждый из них.
- ✅ (2 балла) Затем оценим accuracy для каждого теггера. Заметьте, что в разных системах имена тегов и части речи могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции-конвертера и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции.
- (2 балла) Дальше вам нужно взять лучший теггер для русского языка и с его помощью написать функцию (chunker), которая выделяет из размеченного текста 3 типа n-грамм, соответствующих какому-то шаблону (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд)
- (2-3 балла) В предыдущем дз многие из вас справедливо заметили, что если бы мы могли класть в словарь не только отдельные слова, но и словосочетания, то программа работала бы лучше. Предложите 3 шаблона (слово + POS-тег / POS-тег + POS-тег) запись которых в словарь, по вашему мнению, улучшила бы качество работы программы из предыдущей домашки. Балл за объяснение того, почему именно эти группы вы взяли, балл за встраивание функции в программу из предыдущей домашки, балл за сравнение качества предсказания тональности с улучшением и без (это бонусный одиннадцатый балл).

In [512]:
import os
import re
import csv
from collections import defaultdict

import nltk
from nltk.tokenize import word_tokenize
from corpy.udpipe import Model, load, dump
from razdel import sentenize, tokenize
from slovnet import Morph
from navec import Navec
import pymorphy2
import pandas as pd


pymorph = pymorphy2.MorphAnalyzer()
model = Model('russian-syntagrus-ud-2.5-191206.udpipe')
punctuation = """!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~–—«»"""

## 1. Создание и разметка корпуса

https://docs.google.com/spreadsheets/d/1MdvFcSXRpxD0bcRKDRRP3WSEyUaoXFpWiOFkbpo2j-c/edit#gid=825959713

**Почему этот текст подходит для оценки?**
1. Продуктивные модели: _шиммерный_.
2. Сложные слова: _фан-клуб, лос-анджелесский, апсайкл-деним, утракороткий_.
3. Слова с «неизвестными» корнями без продуктивных аффиксов: _краш, ресейл, инстаграм, апсайклинг, мюли (можно принять за глагол прош. вр. во мн. ч.)_.
4. Редкие и нестандартные формы: _постя_.
5. Куча именованных сущностей: _Эмили Адамс Боде, Жан-Поль Готье_. 

In [333]:
text = """
Кроп-топ Жан-Поль Готье, сумочка Шанель — Беллу Хадид смело можно назвать королевой винтажа.
Но осознанным покупателем ее делает не только пристрастие к ресейлу (помните, надевая 
что-то винтажное, вы сокращаете углеродный след своего гардероба, а еще продлеваете 
жизнь конкретной вещи) — модель постоянно поддерживает молодые независимые марки, 
которые практикуют устойчивость и экологичность, постя фотографии в Инстаграм. 
В фан-клуб Эмили Адамс Боде Белла Хадид вступила в марте 2020-го в самом начале карантина — 
именно тогда в ее инстаграме появилась серия фотографий под кодовым названием «любимый 
желтый жакет». Эмили известна своей страстью к апсайклингу и винтажной стилистике, 
так что их союзу с Беллой удивляться не приходится. В декабре последняя была замечена 
в джемпере Виктория и броских брюках Соуллэнд — очередное подтверждение того, что 
экологичные творения Боде могут быть очень универсальными. Еще один карантинный краш Хадид — 
это расписанные вручную брюки лос-анджелесской художницы Джульет Джонстоун. 
Джонстоун использует старые пары как «холст». Результат — по-настоящему уникальные 
вещи с очень экологичной репутацией. Эстетика нулевых буквально заполонила ленту инстаграма: 
ультракороткие юбки, мюли, шиммерные тени, глиттер. Белла — большая поклонница всего 
винтажного — просто не могла не полюбить Редан (бренд, который обновляет и перепродает 
старые джинсы Левайс). Казалось бы, куда осознаннее, но Редан пошли еще дальше — запустили 
у себя на сайте секцию ресейла. Теперь их апсайкл-деним будет жить и носиться вечно.
"""

**Тегсет**

Выбран на основе статьи "NLP evaluation: russian morphological parsers".
Морфологические признаки указываются только для существительных, глаголов и прилагательных.

- **род**: m (мужской), f (женский), n (средний).
- **падеж**: nom (именительный), gen (родитель- ный, в том числе счетная форма — два шар/а), dat (дательный), acc (винительный), ins (твори- тельный), loc (предложный, в том числе второй предложный, ср. в лесу).
- **число**: sg (единственное), pl (множественное).
- **время**: pres (= непрошедшее: настоящее и будущее время — пишу, напишу), past (прошедшее).
- **наклонение**: imper (повелительное).
- **инфинитив**: inf.
- **причастие**: partcp.
- **деепричастие**: ger.
- **залог**: act (действительный), pass (страдательный) — указывается только в формах причастий.
- **лицо**: 1p, 2p, 3p.

In [431]:
def tokenizer(text):
    return [i for i in nltk.word_tokenize(text) if i not in punctuation]

Загружаю токены в csv, чтобы разметить вручную. 

In [16]:
with open ('HW2_annotated_text.csv', 'a', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    writer.writerow(['token', 'lemma', 'pos', 'gender', 
                      'case', 'num', 'tense', 'mood', 'inf', 
                      'partcp', 'ger', 'voice', 'person'])
    for token in tokenizer(text):
        writer.writerow([token,])

## FAQ

- Как оценивать accuracy, если POS-теггер не разрешает омонимию? - Проверить, что это точно так. За правильные ответы тогда можно считать случаи, когда тег из gold-разметки есть в ответе теггера.

UDPipe: https://corpy.readthedocs.io/en/stable/guides/udpipe.html

## 2. Тестируем POS-теггеры

### 2.1 Pymorphy2

In [432]:
def pymorphy_analysis(text):
    pymorphy_dict = defaultdict()
    tokens = tokenizer(text)
    
    for token in tokens:
        token = token.lower()
        p = pymorph.parse(token)[0]
        pymorphy_dict[token] = {}
        
        if p.tag.POS in ['NOUN', 'VERB']:
            pymorphy_dict[token]['POS'] = p.tag.POS
            
        elif p.tag.POS == 'INFN':
            pymorphy_dict[token]['POS'] = 'VERB'
            pymorphy_dict[token]['inf'] = 'inf'

        elif p.tag.POS in ['PRTF', 'PRTS']:
            pymorphy_dict[token]['POS'] = 'VERB'
            pymorphy_dict[token]['partcp'] = 'partcp'
            
        elif p.tag.POS in ['ADJF', 'ADJS']:
            pymorphy_dict[token]['POS'] = 'ADJ'
        
        elif p.tag.POS == 'GRND':
            pymorphy_dict[token]['POS'] = 'VERB'
            pymorphy_dict[token]['ger'] = 'ger'
            
        else:
            del pymorphy_dict[token]
            continue
        
        #pymorphy_dict[token]['lemma'] = pymorph.parse(token)[0].normal_form
        pymorphy_dict[token]['gender'] = p.tag.gender
        pymorphy_dict[token]['case'] = p.tag.case      
        pymorphy_dict[token]['num'] = p.tag.number 
        pymorphy_dict[token]['tense'] = p.tag.tense
        pymorphy_dict[token]['mood'] = p.tag.mood
        pymorphy_dict[token]['voice'] = p.tag.voice
        pymorphy_dict[token]['person'] = p.tag.person
        
    return pymorphy_dict

### 2.2 Natasha

In [336]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)

In [378]:
def slovnet_analysis(text):
    slovnet_dict = defaultdict()
    
    chunk = [[_.text for _ in tokenize(sent.text)] for sent in sentenize(text)]
    morph.navec(navec)
    for markup in morph.map(chunk):
        for token in markup.tokens:
            text = token.text.lower()
            if token.pos not in ['ADJ', 'NOUN', 'VERB', 'PROPN']:
                continue

            slovnet_dict[text] = {}
            
            if token.pos == 'PROPN':
                slovnet_dict[text]['POS'] = 'NOUN'
            else:
                slovnet_dict[text]['POS'] = token.pos
            
            feats = token.feats    
            slovnet_dict[text]['gender'] = feats.get('Gender', None) 
            slovnet_dict[text]['case'] = feats.get('Case', None)
            slovnet_dict[text]['num'] = feats.get('Number', None)
            slovnet_dict[text]['tense'] = feats.get('Tense', None)
            slovnet_dict[text]['mood'] = feats.get('Mood', None)
            slovnet_dict[text]['voice'] = feats.get('Voice', None)
            slovnet_dict[text]['person'] = feats.get('Person', None)
            try:
                if token.feats['VerbForm'] == 'Inf':
                    slovnet_dict[text]['inf'] = 'inf'
            except KeyError:
                pass
            
            try:
                if token.feats['VerbForm'] == 'Part':
                    slovnet_dict[text]['partcp'] = 'partcp'
            except KeyError:
                pass
            
            try:
                if token.feats['VerbForm'] == 'Conv':
                    slovnet_dict[text]['ger'] = 'ger'
            except KeyError:
                pass
            
    return slovnet_dict

### 2.3 Udpipe

In [383]:
def udpipe_analysis(text):
    udpipe_dict = defaultdict()
    sentences = [s.split('\t')[1:6] for s in "".join(dump(model.process(text))).split('\n') 
                 if re.match(r'\d', s) and s.split('\t')[3] in ['NOUN', 'ADJ', 'VERB', 'PROPN']]
    
    for token_features in sentences:
        token = token_features[0].lower()
        udpipe_dict[token] = {}
        gr_features_dict = {}
        gr_features = token_features[4].split('|')
        for feature in gr_features:
            try:
                gr_features_dict[feature.split('=')[0]] = feature.split('=')[1]
            except IndexError:
                continue
            
        udpipe_dict[token]['POS'] = token_features[2]
        try:
            udpipe_dict[token]['gender'] = gr_features_dict['Gender']
        except KeyError:
            pass
        
        try:
            udpipe_dict[token]['case'] = gr_features_dict['Case']
        except KeyError:
            pass
        
        try:
            udpipe_dict[token]['num'] = gr_features_dict['Number']
        except KeyError:
            pass
        
        try:
            udpipe_dict[token]['tense'] = gr_features_dict['Tense']
        except KeyError:
            pass
        
        try:
            udpipe_dict[token]['mood'] = gr_features_dict['Mood']
        except KeyError:
            pass
        
        try:
            udpipe_dict[token]['voice'] = gr_features_dict['Voice']
        except KeyError:
            pass
        
        try:
            udpipe_dict[token]['person'] = gr_features_dict['Person']
        except KeyError:
            pass
        
        try:
            udpipe_dict[token]['case'] = gr_features_dict['Case']
        except KeyError:
            pass
        
        try:
            if gr_features_dict['VerbForm'] == 'Part':
                udpipe_dict[token]['partcp'] = 'partcp'
        except KeyError:
            pass
        
        try:
            if gr_features_dict['VerbForm'] == 'Inf':
                udpipe_dict[token]['inf'] = 'inf'
        except KeyError:
            pass
        
        try:
            if gr_features_dict['VerbForm'] == 'Conv':
                udpipe_dict[token]['ger'] = 'ger'
        except KeyError:
            pass
   
    return udpipe_dict 

### 2.4 Золотой стандарт

In [342]:
def get_golden_standart(path='HW2_annotated_text.csv'):
    golden_dict = defaultdict()
    with open (path, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        #next(reader, None)
        for token_info in reader:
            if token_info[2] in ['NOUN', 'VERB', 'ADJ']:
                golden_dict[token_info[0]] = {}
                golden_dict[token_info[0]]['POS'] = token_info[2]
                if token_info[3]:
                    golden_dict[token_info[0]]['gender'] = token_info[3]
                if token_info[4]:
                    golden_dict[token_info[0]]['case'] = token_info[4]
                if token_info[5]:
                    golden_dict[token_info[0]]['num'] = token_info[5]
                if token_info[6]:
                    golden_dict[token_info[0]]['tense'] = token_info[6]
                if token_info[7]:
                    golden_dict[token_info[0]]['mood'] = token_info[7]
                if token_info[8]:
                    golden_dict[token_info[0]]['inf'] = token_info[8]
                if token_info[9]:
                    golden_dict[token_info[0]]['partcp'] = token_info[9]
                if token_info[10]:
                    golden_dict[token_info[0]]['ger'] = token_info[10]
                if token_info[11]:
                    golden_dict[token_info[0]]['voice'] = token_info[11]
                if token_info[12]:
                    golden_dict[token_info[0]]['person'] = token_info[12]
                
    return golden_dict        

## 3. Accuracy

Оценим accuracy для каждого теггера.

**Accuracy = P/N**

где P – количество токенов по которым классификатор принял правильное решение, а N – размер обучающей выборки (количество токенок в золотом стандарте). 

In [366]:
pymorphy_dict = pymorphy_analysis(text)
slovnet_dict = slovnet_analysis(text)
udpipe_dict = udpipe_analysis(text)
golden_dict = get_golden_standart()

In [424]:
def get_accuracy(morph_dict, golden_dict):
    tokens = list(map(lambda x: x.lower(), gold_dict.keys()))
    morph_answers = []
    for token in tokens:
        try:
            if morph_dict[token]['POS'] == golden_dict[token]['POS']:
                morph_answers.append(1)
            else:
                morph_answers.append(0)
        except Exception as e:
            continue
            
    return round(sum(morph_answers)/len(gold_dict), 3)

**Accuracy для pymorphy**

In [430]:
get_accuracy(pymorphy_dict, golden_dict)

0.789

**Accuracy для Natasha**

In [429]:
get_accuracy(slovnet_dict, golden_dict)

0.766

**Accuracy для Udpipe**

In [428]:
get_accuracy(udpipe_dict, golden_dict)

0.766

**Выигрывает ✨ Pymorphy ✨**

## 4. Chunker

Дальше вам нужно взять лучший теггер для русского языка (aka Pymorphy) и с его помощью написать функцию (chunker), которая выделяет из размеченного текста 3 типа n-грамм, соответствующих какому-то шаблону (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд)

**3 типа n-грамм:**
- 'не' + VERB
- ADJ + NOUN
- VERB + (ADJ.Acc)? + NOUN.Acc

In [502]:
def get_pattern_NEG_any_POS(tokens):
    pattern_matches = []
    
    for i in range(len(tokens)-1):
        current_token = tokens[i].lower()
        next_token = tokens[i+1].lower()
        if current_token == 'не':
            pattern_matches.append((current_token, next_token))
    
    return pattern_matches     

In [503]:
get_pattern_NEG_any_POS(tokenizer(text))

[('не', 'только'), ('не', 'приходится'), ('не', 'могла'), ('не', 'полюбить')]

In [492]:
def get_pattern_ADJ_NOUN(tokens):
    pattern_matches = []

    for i in range(len(tokens)-1):
        current_token = tokens[i].lower()
        next_token = tokens[i+1].lower()
        current_token_POS = pymorph.parse(current_token)[0].tag.POS
        next_token_POS = pymorph.parse(next_token)[0].tag.POS
        if (current_token_POS == 'ADJF' or current_token_POS == 'ADJS') and next_token_POS == 'NOUN':
            pattern_matches.append((current_token, next_token))
    return pattern_matches

In [493]:
get_pattern_ADJ_NOUN(tokenizer(text))

[('осознанным', 'покупателем'),
 ('углеродный', 'след'),
 ('своего', 'гардероба'),
 ('конкретной', 'вещи'),
 ('независимые', 'марки'),
 ('самом', 'начале'),
 ('кодовым', 'названием'),
 ('желтый', 'жакет'),
 ('своей', 'страстью'),
 ('винтажной', 'стилистике'),
 ('броских', 'брюках'),
 ('очередное', 'подтверждение'),
 ('экологичные', 'творения'),
 ('карантинный', 'краш'),
 ('лос-анджелесской', 'художницы'),
 ('старые', 'пары'),
 ('уникальные', 'вещи'),
 ('экологичной', 'репутацией'),
 ('ультракороткие', 'юбки'),
 ('шиммерные', 'тени'),
 ('большая', 'поклонница'),
 ('старые', 'джинсы')]

In [494]:
def get_pattern_VERB_NOUN_Acc(tokens):
    pattern_matches = []

    for i in range(len(tokens)-2):
        current_token = tokens[i].lower()
        next_token = tokens[i+1].lower()
        
        current_token_POS = pymorph.parse(current_token)[0].tag.POS
        next_token_POS = pymorph.parse(next_token)[0].tag.POS
        next_token_case = pymorph.parse(next_token)[0].tag.case
        
        if current_token_POS == 'VERB' and next_token_POS == 'NOUN' and next_token_case == 'accs':
            pattern_matches.append((current_token, next_token))
    return pattern_matches

In [495]:
get_pattern_VERB_NOUN_Acc(tokenizer(text))

[('продлеваете', 'жизнь'), ('заполонила', 'ленту')]

In [498]:
def chunker(text):
    tokens = tokenizer(text)
    pattern_NEG_VERB = get_pattern_NEG_VERB(tokens)
    patter_ADJ_NOUN = get_pattern_ADJ_NOUN(tokens)
    pattern_VERB_NOUN_Acc = get_pattern_VERB_NOUN_Acc(tokens)
    
    return pattern_NEG_VERB, patter_ADJ_NOUN, pattern_VERB_NOUN_Acc

In [499]:
chunker(text)

([('не', 'приходится'), ('не', 'могла')],
 [('осознанным', 'покупателем'),
  ('углеродный', 'след'),
  ('своего', 'гардероба'),
  ('конкретной', 'вещи'),
  ('независимые', 'марки'),
  ('самом', 'начале'),
  ('кодовым', 'названием'),
  ('желтый', 'жакет'),
  ('своей', 'страстью'),
  ('винтажной', 'стилистике'),
  ('броских', 'брюках'),
  ('очередное', 'подтверждение'),
  ('экологичные', 'творения'),
  ('карантинный', 'краш'),
  ('лос-анджелесской', 'художницы'),
  ('старые', 'пары'),
  ('уникальные', 'вещи'),
  ('экологичной', 'репутацией'),
  ('ультракороткие', 'юбки'),
  ('шиммерные', 'тени'),
  ('большая', 'поклонница'),
  ('старые', 'джинсы')],
 [('продлеваете', 'жизнь'), ('заполонила', 'ленту')])

# 5. Улучшаем прошлую домашку

Предложите 3 шаблона (слово + POS-тег / POS-тег + POS-тег) запись которых в словарь, по вашему мнению, улучшила бы качество работы программы из предыдущей домашки. 

✅ Почему именно эти группы вы взяли.

✅ Встраивание функции в программу из предыдущей домашки.

✅ Сравнение качества предсказания тональности с улучшением и без (это бонусный одиннадцатый балл).

**3 шаблона:**
- 'не' + VERB 
- ADV + VERB 
- ADJ + NOUN 

**Почему именно эти шаблоны?**
_Не_,  наречие перед глаголом и прилагательное перед существительным существенно влияют на смысл словосочетания. 
- Сможем различать _не понравилось / понравилось_ и _не разочаровали / разочаровали_.
- Сможем различать _ужасно понравилось_ и _ужасно разочаровались_.
- Сможем различать _великолепный отель_ и _великолепная пыль_.


### 5.1 Функции для выделения паттернов

In [516]:
def get_pattern_NEG_VERB(tokens):
    pattern_matches = []
    
    for i in range(len(tokens)-1):
        current_token = tokens[i].lower()
        next_token = tokens[i+1].lower()
        if current_token == 'не' and pymorph.parse(next_token)[0].tag.POS in ['INFN', 'GRND', 'PRTF', 'PRTS']:
            pattern_matches.append((current_token, next_token))
    
    return pattern_matches 

In [517]:
# Паттерн ADV + VERB (_очень понравилось, ужасно разочаровались_)
get_pattern_NEG_VERB(tokenizer(text))

[('не', 'полюбить')]

In [518]:
def get_pattern_ADV_VERB(tokens):
    pattern_matches = []

    for i in range(len(tokens)-2):
        current_token = tokens[i].lower()
        next_token = tokens[i+1].lower()
        
        current_token_POS = pymorph.parse(current_token)[0].tag.POS
        next_token_POS = pymorph.parse(next_token)[0].tag.POS
        
        if current_token_POS == 'ADVB' and next_token_POS == 'VERB':
            pattern_matches.append((current_token, next_token))
    return pattern_matches

In [519]:
# Паттерн ADV + VERB (_очень понравилось, ужасно разочаровались_)
get_pattern_ADV_VERB(tokenizer(text))

[('еще', 'продлеваете'),
 ('постоянно', 'поддерживает'),
 ('буквально', 'заполонила')]

In [520]:
# Паттерн ADJ + NOUN (_великолепный отель, великолепная пыль_)
get_pattern_ADJ_NOUN(tokenizer(text))

[('осознанным', 'покупателем'),
 ('углеродный', 'след'),
 ('своего', 'гардероба'),
 ('конкретной', 'вещи'),
 ('независимые', 'марки'),
 ('самом', 'начале'),
 ('кодовым', 'названием'),
 ('желтый', 'жакет'),
 ('своей', 'страстью'),
 ('винтажной', 'стилистике'),
 ('броских', 'брюках'),
 ('очередное', 'подтверждение'),
 ('экологичные', 'творения'),
 ('карантинный', 'краш'),
 ('лос-анджелесской', 'художницы'),
 ('старые', 'пары'),
 ('уникальные', 'вещи'),
 ('экологичной', 'репутацией'),
 ('ультракороткие', 'юбки'),
 ('шиммерные', 'тени'),
 ('большая', 'поклонница'),
 ('старые', 'джинсы')]

### 5.2 Встраиваю функции в прошлую домашку

См. [HW_1.ipynb](https://github.com/soimmary/NLP_HW/blob/main/HW_1/HW_1.ipynb)

**Спойлер**: accuracy определения положительных отзывов увеличилось в 2 раза!